# Thoth 0.5.0 - Example 2 Guided Notebook - Runtime Environment

This notebook is a supportive material for example 2 of Thoth's 0.5.0 release. The prerequisite for this notebook is "Thoth 0.5.0 - Example 1 Guided Notebook" which will cover some of the commands more in depth. This notebook expects the reader is familiar with Example 1 Guided Notebook.


See internal document for more info and clarification in [Google Docs](https://docs.google.com/document/d/1QflQpGXtOuHFFC2hkEFBlu0JmCQWEnXdgryvwxCNXpQ/edit#) - section "Example 2".

This notebook is an in-depth description on what Thoth does on lower level. From user's perspective a user just installs `Thamos`, adjusts configuration via `thamos config` (automatic discovery of available hardware is performed) and issues `thamos advise`. [Follow README instructions in thamos repo](https://github.com/thoth-station/thamos/tree/master/examples/runtime-environment) to experience this on your own.


## Initial graph database setup

In order to go through this scenario, we need first connect to a graph database instance. This notebook is playeble from within your computer, it inserts all the data into a provided JanusGraph instance, so select a graph database instance you would like to use. If you want to run this script purely on your local machine, setup your local graph database as [described in the README file of janusgraph-thoth-config repo](https://github.com/thoth-station/janusgraph-thoth-config#running-janusgraph-instance-locally). Ideally, just clone the repo and issue the following command to setup your local JanusGraph database instance:

```
sudo ./local.sh all
```

In [1]:
# Configure JanusGraph instance to talk to:
JANUSGRAPH_SERVICE_HOST = 'localhost'

# For directly talking to test environment, uncomment the following line:
#JANUSGRAPH_SERVICE_HOST = 'janusgraph.test.thoth-station.ninja'

Now let's connect to desired JanusGraph database and check if we are properly connected:

In [2]:
from thoth.storages import GraphDatabase

# Instantiate and connect the JanusGraph database.
graph = GraphDatabase.create(JANUSGRAPH_SERVICE_HOST)
graph.connect()

graph.is_connected()

True

In the next step we obtain data which were computed by [thoth-solver](https://github.com/thoth-station/solver) - all the dependencies of TensorFlow (direct as well as the transitive ones). This resolution was done against multiple Python package indexes. Besides the upstream one, [pypi.org](https://pypi.org), there were also resolved dependencies of packages hosted on the [AICoE Python package index](https://tensorflow.pypi.thoth-station.ninja/). This index, as of now, hosts optimizes TensorFlow releases which are built for a specific operating system with specific TensorFlow configuration (see cooperation with [Google's TensorFlow on-demand project](https://github.com/tensorflow/community/pull/69)) - in fact its an index of indexes following [PEP-503](https://www.python.org/dev/peps/pep-0503/) standard:

In [3]:
import requests
from thoth.common import timestamp2datetime

SOLVER_DOCUMENT_URL = 'https://raw.githubusercontent.com/thoth-station/misc/master/examples/runtime-environment/resolved.json'

response = requests.get(SOLVER_DOCUMENT_URL)
response.raise_for_status()
solver_document = response.json()

print("Solver resolved stacks for %r" % solver_document["metadata"]["arguments"]["pypi"]["requirements"].replace("\\n", "\n"))
print("Stacks were resolved at", timestamp2datetime(solver_document["metadata"]["timestamp"]))

Solver resolved stacks for 'tensorflow'
Stacks were resolved at 2019-03-06 14:19:03+00:00


In the next step, we show what operating system (environment) and what Python version was used when resolving software stacks. As Python is a dynamic programming language, [resolution of dependencies highly depends on evaluation on a package installation](https://dustingram.com/articles/2018/03/05/why-pypi-doesnt-know-dependencies/). That's one of the main reasons why Thoth pre-computes dependencies and creates structures inside graph database to have notion about ecosystem structure (another significant reason is speed of generating software stacks when scoring, installations of Python packages is slow). This pre-computation is done by solvers which are installed in a Thoth deployment (they can be dynamically added, Thoth automatically resolves monitored packages in the ecosystem and tracks updates).

In [4]:
from  thoth.storages import SolverResultsStore

document_id = SolverResultsStore.get_document_id(solver_document)
solver_name = SolverResultsStore.get_solver_name_from_document_id(document_id)
print("Solver configuration was:\n\t%r" % graph.parse_python_solver_name(solver_name))

Solver configuration was:
	{'os_name': 'fedora', 'os_version': '28', 'python_version': '3.6'}


In [5]:
indexes_seen = set()
for item in solver_document["result"]["tree"]:
    indexes_seen.add(item["index_url"])

print("All the indexes which were resolved during solver scan:")
print("\n".join(indexes_seen))

All the indexes which were resolved during solver scan:
https://tensorflow.pypi.thoth-station.ninja/index/centos7/jemalloc/simple
https://tensorflow.pypi.thoth-station.ninja/index/fedora27/jemalloc/simple
https://pypi.org/simple
https://tensorflow.pypi.thoth-station.ninja/index/fedora28/jemalloc/simple


All the indexes are added in Thoth's deployment in an initialization job called [init-job](https://github.com/thoth-station/init-job) after a fresh provisioning. The job scans the AICoE index for configured indexes and inserts them into Thoth's database. As a Thoth operator, one can add new indexes on management API endpoints as desired. Thoth will automatically scan these indexes for packages and resolve software stacks with registered solvers.

If there is any new release of a package, Thoth automatically runs thoth-solver for new package releases on [PEP-503 compatible package sources](https://www.python.org/dev/peps/pep-0503/). If you deploy [Warehouse](https://warehouse.readthedocs.io/) on your own, Thoth can be configured to talk directly to its REST API which optimizes some of the requests performed by Thoth components.
Let's sync results of this solver run to the graph database:

In [6]:
%%time

graph.sync_solver_result(solver_document)

Cannot sync unsolvable package {'index': 'https://tensorflow.pypi.thoth-station.ninja/index/rhel7.5/jemalloc/simple', 'package_name': 'tensorflow', 'version_spec': '>=0.0.0'} as package is not locked to as specific version
Cannot sync unsolvable package {'index': 'https://tensorflow.pypi.thoth-station.ninja/index/fedora26/jemalloc/simple', 'package_name': 'tensorflow', 'version_spec': '>=0.0.0'} as package is not locked to as specific version
Cannot sync unsolvable package {'index': 'https://tensorflow.pypi.thoth-station.ninja/index/rhel7.5/cuda9.2+jemalloc/simple', 'package_name': 'tensorflow', 'version_spec': '>=0.0.0'} as package is not locked to as specific version


CPU times: user 4min 54s, sys: 15.2 s, total: 5min 9s
Wall time: 33min 49s


As we want to show guidenance based on performance index, besides solver document, we need to sync also `inspection` documents which gather runtime specific characteristics for user's application stacks which were computed on [Amun](https://github.com/thoth-station/amun-api/).

Amun is a service which validates software stacks - whether they do or do not build on the given buildtime environment and if yes, whether they run on the desired runtime environment. A user (a human being or system) has also ability to supply a script (testing application) which can be used to obtain performance related indicators. An example of such script can be [matrix multiplication](https://github.com/thoth-station/performance/blob/master/tensorflow/matmul.py) - results of this script will be used in the upcoming paragraphs.

![Amun service](https://raw.githubusercontent.com/thoth-station/amun-api/master/fig/diagram.png "Amun service")

Let's obtain results of Amun's job runs - in Thoth's universe, these runs are called "inspections" and let's sync them into graph database (in a deployment, these documents are automatically gathered and stored in Ceph, the sync is subsequently done via [graph-sync-job](https://github.com/thoth-station/graph-sync-job) scheduled by [graph-sync-scheduler](https://github.com/thoth-station/graph-sync-scheduler) and run by [workload-operator](https://github.com/thoth-station/workload-operator)).

The main aim of an inspection is to gather application stack characteristics, such as:

* Can be the given stack assembled?
  * Gather errors observed during software stack installation.

* Is application runnable?
  * Gather errors done due to issues, such as ABI incompatibility or Python version requirements.

The results of characteristics above are automatically synced into Thoth's knowledge base.

To perform validation of different software stacks depending on package versions used, [Dependency Monkey](http://thoth-station.ninja/docs/developers/adviser/dependency_monkey.html#dependency-monkey) was used. Dependency Monkey is able to generate stacks and submit them to Amun for inspection in the given runtime/buildtime environment. You can find results of Dependency Monkey runs on Ceph, where Thoth primarily stores them and subsequently syncs them to the graph database. However, in this Jupyter Notebook we will use inspection documents stored on GitHub so anyone (without access to Ceph) can run this notebook and experiment with data:

In [7]:
%%time
import requests
import json

INSPECTION_DOCUMENT_BASE_URL = 'https://raw.githubusercontent.com/thoth-station/misc/master/examples/runtime-environment/inspection_%d.json'

inspections = []
for inspection_id in range(1, 5):
    url = INSPECTION_DOCUMENT_BASE_URL % inspection_id
    response = requests.get(url)
    response.raise_for_status()
    inspection_document = response.json()
    print("Syncing document %r into graph database..." % inspection_document["inspection_id"])
    if "performance_index" in inspection_document["job_log"]["stdout"]:
        print("  performance index: %f" % inspection_document["job_log"]["stdout"]["performance_index"])
    graph.sync_inspection_result(inspection_document)
    inspections.append(inspection_document)

Syncing document 'inspect-002c9fbb7513b20b' into graph database...
  performance index: 23.123200
Syncing document 'inspect-003e9fc7135f503f' into graph database...
  performance index: 12.322200
Syncing document 'inspect-0044bd85b0d234cb' into graph database...
  performance index: 22.333333
Syncing document 'inspect-36b2a777b71d3fff' into graph database...
CPU times: user 780 ms, sys: 86.7 ms, total: 867 ms
Wall time: 3.82 s


Let's have a closer look at inspection document:

Full `build_log` is carried for additional analysis - all the tools (such as `pip`, `Pipenv`, ...) were run in verbose mode to gather as much information as possible. These data are unstructured (raw string). The `job_log` holds a JSON which gathers all the hardware related information about a node on which the inspection job was run (see [amun-hwinfo](https://github.com/thoth-station/amun-hwinfo) init container run on each inspection job). In the the job log, one can find additional information obtained from the script which was run (such as duration of an inspection job, process exit code, error and debug information, ...). Newer versions of amun-hwinfor obtain also information about Linux kernel used together with information about CUDA which is present:

In [8]:
inspection_document["job_log"]

{'exit_code': -6,
 'hwinfo': {'cpu': {'has_3dnow': False,
   'has_3dnowext': False,
   'has_Altivec': None,
   'has_f00f_bug': None,
   'has_fdiv_bug': None,
   'has_mmx': True,
   'has_sse': True,
   'has_sse2': True,
   'has_sse3': True,
   'has_ssse3': True,
   'is_32bit': False,
   'is_64bit': True,
   'is_AMD': False,
   'is_AMD64': False,
   'is_Alpha': None,
   'is_Athlon64': False,
   'is_AthlonHX': False,
   'is_AthlonK6': False,
   'is_AthlonK6_2': False,
   'is_AthlonK6_3': False,
   'is_AthlonK7': False,
   'is_AthlonMP': False,
   'is_Celeron': False,
   'is_Core2': False,
   'is_EV4': None,
   'is_EV5': None,
   'is_EV56': None,
   'is_Hammer': False,
   'is_Intel': True,
   'is_Itanium': None,
   'is_Nocona': False,
   'is_Opteron': False,
   'is_PCA56': None,
   'is_Pentium': False,
   'is_PentiumII': False,
   'is_PentiumIII': False,
   'is_PentiumIV': False,
   'is_PentiumM': False,
   'is_PentiumMMX': False,
   'is_PentiumPro': False,
   'is_Power': None,
   'is_Powe

To make an inspection document a full report, it carries also all the information which was sent to Amun (from Dependency Monkey or directly with API call). The inspection "specification" (holds all the information about hardware required, Python packages used - fully pinned down stack, script used to perform performance related measurements, ...):

In [9]:
inspection_document["specification"]

{'base': 'fedora:28',
 'build': {'requests': {'cpu': '500m',
   'hardware': {'cpu_family': 6,
    'cpu_model': 94,
    'physical_cpus': 32,
    'processor': 'Intel Core Processor (Skylake, IBRS)'},
   'memory': '1Gi'}},
 'files': None,
 'packages': ['pipenv', 'which', 'python36'],
 'python': {'requirements': {'dev-packages': {},
   'packages': {'absl-py': {'index': 'pypi-org', 'version': '==0.6.1'},
    'astor': {'index': 'pypi-org', 'version': '==0.7.1'},
    'bleach': {'index': 'pypi-org', 'version': '==1.5.0'},
    'gast': {'index': 'pypi-org', 'version': '==0.2.0'},
    'html5lib': {'index': 'pypi-org', 'version': '==1.0.1'},
    'markdown': {'index': 'pypi-org', 'version': '==3.0.1'},
    'olefile': {'index': 'pypi-org', 'version': '==0.42.1'},
    'pillow': {'index': 'pypi-org', 'version': '==5.0.0'},
    'setuptools': {'index': 'pypi-org', 'version': '==40.6.2'},
    'six': {'index': 'pypi-org', 'version': '==1.9.0'},
    'tensorboard': {'index': 'pypi-org', 'version': '==1.10.0

The last inspection on index 3 has negative observation we observed in inspection runs (broken ABI):

In [10]:
print("stderr:\n\t%s" % inspections[3]["job_log"]["stderr"])
print("exit code:", inspections[3]["job_log"]["exit_code"])

stderr:
	2019-01-22 19:27:02.397499: F tensorflow/python/lib/core/bfloat16.cc:664] Check failed: PyBfloat16_Type.tp_base != nullptr 

exit code: -6


Now, we have all the data needed available in the graph database. Let's perform scoring based on data we have gathered and synced.

## Scoring

Let's suppose we run a machine learning application using TensorFlow and we would like to ask Thoth for the best TensorFlow stack we could use which works and performs well on the given runtime environment with specific hardware configuration.

Let's use [Pipenv](https://pipenv.readthedocs.io) for dependency managemnet and let's ask Thoth's recommendation engine, [thoth-adviser](https://github.com/thoth-station/adviser), what is the best application stack we should use.

In [11]:
import requests

PIPFILE_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/runtime-environment/Pipfile"
PIPFILE_LOCK_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/runtime-environment/Pipfile.lock"

response = requests.get(PIPFILE_URL)
response.raise_for_status()
pipfile_str = response.text

response = requests.get(PIPFILE_LOCK_URL)
response.raise_for_status()
pipfile_lock_str = response.text

Our application has only one direct dependency - TensorFlow:

In [12]:
print(pipfile_str)

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
absl-py = "==0.6.1"
astor = "==0.7.1"
bleach = "==1.5.0"
gast = "==0.2.0"
markdown = "==3.0.1"
olefile = "==0.42.1"
pillow = "==5.0.0"
tensorflow = "==1.9.0"
webencodings = "==0.5.1"
werkzeug = "==0.14.1"



In [13]:
pipfile_str = """
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
absl-py = "==0.6.1"
astor = "==0.7.1"
bleach = "==1.5.0"
gast = "==0.2.0"
#grpcio = "==1.9.1"
html5lib = "==1.0.1"
markdown = "==3.0.1"
#numpy = "==1.15.2"
olefile = "==0.42.1"
pillow = "==5.0.0"
#protobuf = "==3.6.1"
setuptools = "==40.6.2"
six = "==1.9.0"
tensorboard = "==1.10.0"
tensorflow = "==1.9.0"
termcolor = "==1.1.0"
webencodings = "==0.5.1"
werkzeug = "==0.14.1"
wheel = "==0.32.3"
"""

The fully pinned down software stack, however, consists of all the transitive dependencies TensorFlow has:

In [14]:
print(pipfile_lock_str)

{
    "_meta": {
        "hash": {
            "sha256": "42daab12bab8993c4f7d12e3e0ee615c70deb2d6cdfa1f291899aca6da31b367"
        },
        "pipfile-spec": 6,
        "requires": {},
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "absl-py": {
            "hashes": [
                "sha256:87519e3b91a3d573664c6e2ee33df582bb68dca6642ae3cf3a4361b1c0a4e9d6"
            ],
            "index": "pypi",
            "version": "==0.6.1"
        },
        "astor": {
            "hashes": [
                "sha256:95c30d87a6c2cf89aa628b87398466840f0ad8652f88eb173125a6df8533fb8d",
                "sha256:fb503b9e2fdd05609fbf557b916b4a7824171203701660f0c55bbf5a7a68713e"
            ],
            "index": "pypi",
            "version": "==0.7.1"
        },
        "backports.weakref": {
            "hashes": [
                "sha256:

As we are querying Thoth's knowledge base for a specific runtime environment, we need to provide additional information to Thoth so we can retrieve the best possible application stack for our environment. These information do not need to be provided by hand, Thoth's CLI utility [Thamos](https://github.com/thoth-station/thamos) can perform automatic detection of hardware and operating system in a deployment and send this information to Thoth:


```
# Perform automatic detection of hardware, operating system and software available (e.g. CUDA):
thamos config --no-interactive
# Ask Thoth for the best application stack:
thamos advise
# Install Python packages into the given environemnt (e.g. s2i):
pipenv install
```

To demonstrate the scenario from within this Jupyter Notebook, we will use prepared Thoth's configuration:

In [15]:
import yaml
import requests

THOTH_YAML_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/runtime-environment/.thoth.yaml"

response = requests.get(THOTH_YAML_URL)
response.raise_for_status()
content = response.text
content = content.replace("85", "94")
content = content.replace("29", "28")
print(content)
config_content = yaml.load(content)

#
# This is a default Thoth configuration file placed in a root of a repo. Please adjust values listed below.

host: test.thoth-station.ninja
tls_verify: false

recommendation_type: stable
requirements_format: pipenv

runtime_environments:
  - name: "fedora:28"
    # Operating system for which the recommendations should be created:
    operating_system:
      name: fedora
      version: "28"
    # Hardware information for the recommendation engine:
    hardware:
      cpu_family: 6
      cpu_model: 94
    # Software configuration of runtime environment:
    python_version: "3.6"
    cuda_version: null
    # Recommendation type - one of testing, stable, latest:
    recommendation_type: stable
    limit_latest_versions: null



The upcoming step initializes `RuntimeEnvironment` for Thoth's `Project` abstraction which will be passed to adviser:

In [ ]:
from thoth.common import RuntimeEnvironment

# We will stick with the first runtime environment stated in the configuration file (there is one stated anyway).
runtime_environment = RuntimeEnvironment.from_dict(config_content["runtime_environments"][0])

Unknown configuration entry in the configuration file recommendation_type with value stable
Unknown configuration entry in the configuration file limit_latest_versions with value None


In [ ]:
from thoth.python import Project

project = Project.from_strings(pipfile_str, pipfile_lock_str, runtime_environment=runtime_environment)
project.to_dict()

{'requirements': {'packages': {'absl-py': '==0.6.1',
   'astor': '==0.7.1',
   'bleach': '==1.5.0',
   'gast': '==0.2.0',
   'html5lib': '==1.0.1',
   'markdown': '==3.0.1',
   'olefile': '==0.42.1',
   'pillow': '==5.0.0',
   'setuptools': '==40.6.2',
   'six': '==1.9.0',
   'tensorboard': '==1.10.0',
   'tensorflow': '==1.9.0',
   'termcolor': '==1.1.0',
   'webencodings': '==0.5.1',
   'werkzeug': '==0.14.1',
   'wheel': '==0.32.3'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi'}]},
 'requirements_locked': {'_meta': {'sources': [{'url': 'https://pypi.org/simple',
     'verify_ssl': True,
     'name': 'pypi'}],
   'requires': {},
   'hash': {'sha256': '2aa54a21b98a8554f29f4679b2c77248f95b82506939b73322af9b5ed9ec4a19'},
   'pipfile-spec': 6},
  'default': {'absl-py': {'version': '==0.6.1',
    'hashes': ['sha256:87519e3b91a3d573664c6e2ee33df582bb68dca6642ae3cf3a4361b1c0a4e9d6'],
    'index': 'pypi'},
   'astor': {'vers

The warning reported can be omitted. The `recommendation_type` configuration option in the configuration file is used in Thamos to override default `recommendation_type` if needed (for a specific runtime environment entry).

Everything set, let's finally compute some advises:

In [ ]:
%%time
%env THOTH_ADVISER_SHOW_PACKAGES=1
%env ISIS_API_URL=http://isis.test.thoth-station.ninja

from thoth.adviser.python import Adviser
from thoth.adviser.enums import RecommendationType

stack_info, advised_configuration, report = Adviser.compute_on_project(
    project,
    recommendation_type=RecommendationType.STABLE,
    count=10,                      # Number of stacks reported in the output.
    limit=None,                   # Number of stacks scored in total.
    limit_latest_versions=None,   # Number of latest versions for each package considered (at most).
    graph=graph,
    dry_run=False,
)

env: THOTH_ADVISER_SHOW_PACKAGES=1
env: ISIS_API_URL=http://isis.test.thoth-station.ninja


2019-03-21 09:39:01,111 [2131] INFO     root:126: Logging to a Sentry instance is turned off
2019-03-21 09:39:01,112 [2131] INFO     root:148: Logging to rsyslog endpoint is turned off
2019-03-21 09:39:01,123 [2131] INFO     thoth.adviser.python.scoring:56: Using scoring function obtaining stable software stacks
2019-03-21 09:39:01,123 [2131] INFO     thoth.adviser.python.dependency_graph:448: Parsing and solving direct dependencies of the requested project
2019-03-21 09:39:01,124 [2131] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'absl-py==0.6.1'
2019-03-21 09:39:01,768 [2131] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'astor==0.7.1'
2019-03-21 09:39:02,419 [2131] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'bleach==1.5.0'
2019-03-21 09:39:03,100 [2131] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'gast==0.2.0'
2019-03-21 09:39:03,733 [2131] INFO     thoth.solver.python.python_solver:113: Parsi

2019-03-21 09:39:48,243 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.14.1' from https://pypi.org/simple
2019-03-21 09:39:48,244 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.14.2' from https://pypi.org/simple
2019-03-21 09:39:48,245 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.15.0' from https://pypi.org/simple
2019-03-21 09:39:48,246 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.16.0' from https://pypi.org/simple
2019-03-21 09:39:48,247 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.16.1' from https://pypi.org/simple
2019-03-21 09:39:48,247 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.17.0' from https://pypi.org/simple
2019-03-21 09:39:48,247 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.17.1' from https://pypi.org/simple
2019-03-21 09:39:48,248 [2131] INFO     thoth.adviser.python.dependency_graph:659: 	'1.18.0' from https://pypi.org/simple
2019-03-21 09:39:48,248 

2019-03-21 09:39:54,546 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,617 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,684 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,751 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,819 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,890 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:54,958 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:55,028 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


2019-03-21 09:39:58,961 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,038 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,116 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,180 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,254 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,326 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,390 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:39:59,459 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


...

2019-03-21 09:42:27,378 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,472 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,544 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,614 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,684 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,759 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,835 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:27,922 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


2019-03-21 09:42:32,427 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,488 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,553 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,619 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,685 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,754 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,818 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:32,883 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


2019-03-21 09:42:37,656 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:37,753 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:37,840 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:37,921 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:38,005 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:38,082 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:38,165 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:38,250 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


2019-03-21 09:42:43,358 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,442 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,508 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,573 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,643 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,712 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,780 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:43,843 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


2019-03-21 09:42:48,017 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,082 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,142 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,222 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,287 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,352 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,420 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0
2019-03-21 09:42:48,480 [2131] INFO     thoth.adviser.python.dependency_graph:758: Found a new stack with total score of 0.0


In [ ]:
report[0][1].pipfile_lock.to_dict()

In [ ]:
report[0][0]

Parameter `count` limits number of stacks provided in the output, parameter `limit` limits numbef of stacks scored in total. Parameter `limit_latest_versions` limits number of latest versions for each package to consider (for example consider first 4 latest releases of `numpy`). 

To understand what is going on on the background, lets dive into steps in the following chapters.

All the direct dependencies are parsed and resolved given the version ranges stated in `Pipfile`. The resolution is done offline but respecting `pip`'s logic (`pip`'s internal resolution logic is reused), based on Thoth's resolver implemented on top of the graph instance (resolution is thus faster and respects packages known to Thoth - packages already analyzed). This resolution is done cross-package source index, meaning you can configure Thoth to be a resolver on top of different Python package indexes. This is something some tools partially support (such as [Pipenv](https://pipenv.readthedocs.io/en/latest/advanced/#specifying-package-indexes)), however in Python ecosystem, all package sources are treated as mirrors. If a package cannot be found on one package index, current tools silently fallback to another Python package index which can be miss-leading or undesired in some cases (e.g. if you want to consume optimized TensorFlow wheel files from AICoE index, you don't want to fallback to an un-optimized upstream TensorFlow release, see Thoth's provenance checks for more info and for provenance gating). This issue is present also in `Pipenv` even though the docs do not state it.

Once the resolution is done on direct dependencies, there are issued queries to JanusGraph instance to gather transitive dependencies of all the direct dependencies in a specific version from a specific package source index respecting their version ranges (on specific environment, in this case Fedora 29 with Python 3.6). The result of query is in fact, a serialized dependency graph as stored in the Thoth's graph database.

As some of the packages are NOT installable into the given environment (e.g. too old versions which do not run on Python 3+, but run only on legacy Python 2) are automatically removed in a cut-off phase. This way we reduce stacks we know they will not work at all and reduce space traversed during stacks generation. Another type of packages which are removed in the cut-off phase are "core" Python packages which we pick automatically to the latest working release (e.g. `setuptools`, `pip`, ...).

After the step above is done, Thoth's adviser implementation sorts packages in the resulting query in such way, they can be used to generate software stacks systematically, from the latest stack to the oldest one. This way, there are prefered newer stacks in comparision to the older ones - the newer stack is, the sooner it gets scored and takes precedence in case of same score with another but older software stack.

If a user supplies `limit_latest_versions` option, there are removed older versions of each package (in the example above, there are considered only 4 latest versions of each package due to memory requirements). This option helps to reduce space - for larger stacks, number of stacks result in a really huge number and the depedency graph traversal does not fit into memory (you can try it by yourself by removing limit_latest_versions parameter or setting it to None). See estimation in the adviser log to check number of stacks estimated for scoring - `estimated stacks count` (its an estimation - upper bound, the actual number may, and usually will, differ based on the dependency graph traversal).

Now, we are ready to perform the actual graph traversal. The current implemenation uses a subprocess which calls to C/C++ dependency graph implementation implemented in a form of a library. The subprocess generates a stream of stacks which is consumed by the parent process (see [docs for more info](http://thoth-station.ninja/docs/developers/adviser/libdependency_graph.html)) to perform stack scoring. As stated above, there are generated stacks from the newest one to the oldest one. The actual scoring is performed based on data stored in the graph database.

In this Jupyter Notebook we show "`STABLE`" recommendation type which covers performance based scoring, data for this scoring are already present in the graph database and are derived from `inspection` runs. To get familiar with other types of scoring see other Thoth notebooks (e.g. CVE based scoring in the third example notebook).

The scoring is computed as an average of performance related inspection runs, where inspections which show that the stack does not work have negative (-1.0) performance score, where positive performance scores reflect performance score in inspections (see [thoth-station/performance](https://github.com/thoth-station/performance/) for performance indicators used).

As some of the packages do not have performance impact on a software stack, Thoth uses Isis and its feature based queries on top of [project2vec](https://docs.google.com/document/d/18Nghre5s4O8MUvslU2n_avznEmF78gOiDq_Xe6ygMjc). In this case the query consists of only "core performance" packages (for example without `markdown` package in case of TensorFlow stacks) which do affect performance in any way. Thoth internally queries [Isis API](https://github.com/thoth-station/isis-api/) (which is an API for project2vec vector space). If you would like to suppress this behaviour and rather ask for exact match on software stack, just omit `ISIS_API_URL` environment variable - stacks will be scored based on exact match in the graph database.

In [ ]:
%env ISIS_API_URL=http://isis.test.thoth-station.ninja

from thoth.adviser.isis import Isis

isis = Isis()

print("TensorFlow performance impact is", isis.get_python_project_performance_import("tensorflow"))
print("Markdown performance impact is", isis.get_python_project_performance_import("markdown"))
print("protobuf performance impact is", isis.get_python_project_performance_import("protobuf"))

... And let's have a look at the recommended stack (Pipfile and Pipfile.lock):

In [ ]:
report[0][1].pipfile.to_dict()

In [ ]:
report[0][1].pipfile_lock.to_dict()

In [ ]:
report[0][0]

Thoth recommended to use optimized AICoE TensorFlow release. Based on data stored in Thoth, it has the best performance on the given CPU, respecting operating system and Python version we decided to use.

There were also found some additional stack related information (e.g. recommendations on Thoth's configuration or tooling, ...):

In [ ]:
stack_info

In [ ]:
advised_configuration is None  # No changes in Thoth's configuration.

If we take a closer look on software stacks and what is the difference making this performance impact, we can see that they differ in the following packages (their versions):

In [ ]:
import pandas as pd

different_versions = {
    "grpcio": [],
    "numpy": [],
    "protobuf": [],
    "tensorflow": [],
}

for idx, inspection_document in enumerate(inspections):
    python_locked = inspection_document["specification"]["python"]["requirements_locked"]["default"]
    for package_name in different_versions.keys():
        different_versions[package_name].append(python_locked[package_name]["version"][len("=="):])

pd.DataFrame(data=different_versions, index=tuple(f"inspection_{i}" for i in range(1, 5)))

In [ ]:
list(sorted([i[0] for i in graph.get_all_versions_python_package("numpy")]))

The error reported by TensorFlow found in inspection 4 is a known issue when running TensorFlow with a specific numpy version - [see this thread](https://github.com/eldar/pose-tensorflow/issues/68).

In [ ]:
print("stderr:\n%s" % inspections[3]["job_log"]["stderr"])

Feel free to experiment with parameters to adviser:

In [ ]:
# Set recommendation type to one of the following:

[e.name for e in RecommendationType]

In [ ]:
# Adjust runtime specific information (e.g. provide CPU, CUDA information, ...)

RuntimeEnvironment.from_dict({}).to_dict(without_none=False)

In [ ]:
# Adjust version ranges of Python packages being installed. These versions are resolved using
# pip's internal algorithm, so anything which is compatible with PEP-440 (and Pipfile compatible
# for Pipfile inputs) works out of box. Note this resolution is not done by installing
# dependencies (as in case of Pip/Pipenv), but there is implemented resolver on top of
# graph database which can resolve dependencies much faster as all the data are pre-computed.

PIPFILE_STR = """
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

# This can resolve in large number of stacks when running against test database.
[packages]
tensorflow = "==1.9.0"
numpy = "<=1.15.3,>1.9"

[requires]
python_version = "3.6"
"""

project = Project.from_strings(PIPFILE_STR)
project.to_dict()

# Mind dependencies resolved in solver run, unknown dependencies to Thoth, obviusly, cannot be resolved by Thoth.

Happy hacking! ;-)

In [ ]:
from thoth.lab import packages_info

# Let's state Thoth's package versions for reproducible next runs of this Jupyter Notebook.
packages_info()